# Tableau Assignment - Citi Bike Analytics
----

### Summary
This notebook shows the process to clean, explore, merge and analyze the data sets
with information for the `New York Citi Bike Program` from `2017`.


__Note__. The csv files were not saved in this repository because they are very large, 
but you can download them from https://s3.amazonaws.com/tripdata/index.html

----

In [70]:
# Dependencies
import pandas as pd
import datetime
import math
import numpy as np

In [89]:
# Function to calculate distance between latitude longitude pairs with Python -  haversine.py
# Available in https://gist.github.com/rochacbruno/2883505

# Function to calculate the distance between two points using coordinates
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3956 # for miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

### __Explore data sets from 2017__

Took January to define the variables for Tableau

In [71]:
# Read file and print the number of records and data types
csv_name = "./2017/201701.csv"
jan_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Jan2017 : " + str(jan_df.count()))

Records Jan2017 : Trip Duration              726676
Start Time                 726676
Stop Time                  726676
Start Station ID           726676
Start Station Name         726676
Start Station Latitude     726676
Start Station Longitude    726676
End Station ID             726676
End Station Name           726676
End Station Latitude       726676
End Station Longitude      726676
Bike ID                    726676
User Type                  723483
Birth Year                 697600
Gender                     726676
dtype: int64


In [77]:
# Find missing values to define what to do with them
percent_missing = jan_df.isnull().sum() * 100 / len(jan_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Birth Year,4.001233
User Type,0.439398
Trip Duration,0.000000
Start Time,0.000000
Stop Time,0.000000
Start Station ID,0.000000
Start Station Name,0.000000
Start Station Latitude,0.000000
Start Station Longitude,0.000000
End Station ID,0.000000


In [78]:
# Replace missing values for birth year with the median by gender
jan_df['Birth Year'] = jan_df.groupby(['Gender'])['Birth Year'].transform(lambda x: x.fillna(x.median()))

In [80]:
# Replace missing values for user type as if they were Customers
# Only because is a very small percentage and it is assumed that if they were Suscribers the field will not be empty
jan_df['User Type'] = jan_df['User Type'].transform(lambda x: x.fillna("Customer"))

In [81]:
# Verify that there are not more missing values
percent_missing = jan_df.isnull().sum() * 100 / len(jan_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Trip Duration,0.0
Start Time,0.0
Stop Time,0.0
Start Station ID,0.0
Start Station Name,0.0
Start Station Latitude,0.0
Start Station Longitude,0.0
End Station ID,0.0
End Station Name,0.0
End Station Latitude,0.0


In [82]:
# Add columns to make transformations
jan_df.insert(0, 'year', 2017)
jan_df.insert(1, 'month', 'Jan')
jan_df.insert(3, 'tripdurmin', 0)
jan_df.insert(5, 'starthour', 0)
jan_df.insert(6, 'weekday', 0)
jan_df.insert(16, 'distance', 0)
jan_df.insert(21, 'age', 0)
jan_df.insert(22, 'sgender', '')
jan_df.insert(23, 'season', 'Winter')
jan_df.insert(24, 'mileage', 0)

In [83]:
# Transform the values for gender
jan_df.loc[jan_df['Gender'] == 0, 'sgender'] = 'Unknown'
jan_df.loc[jan_df['Gender'] == 1, 'sgender'] = 'Male'
jan_df.loc[jan_df['Gender'] == 2, 'sgender'] = 'Female'

In [84]:
# Calculate the age of the person considering that the year was 2017
jan_df['age'] = 2017 - jan_df['Birth Year']

In [85]:
# Transform the durantion of the trip from seconds to minutes
jan_df['tripdurmin'] = jan_df['Trip Duration'] / 60

In [86]:
# Extact the hour fron the startime
jan_df['starthour'] = pd.to_datetime(jan_df["Start Time"]).dt.strftime('%H')

In [87]:
# Calculate the weekday fron the startime
jan_df['weekday'] = pd.to_datetime(jan_df["Start Time"]).dt.strftime('%A')

In [90]:
# Calculate the distance in miles from start station to end station
jan_df['distance'] = jan_df.apply(lambda row: distance((row['Start Station Latitude'],row['Start Station Longitude']), 
                         (row['End Station Latitude'], row['End Station Longitude'])),
                         axis=1)

In [91]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
jan_df.loc[jan_df['tripdurmin'] <= 120, 'mileage'] = (jan_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
jan_df.loc[jan_df['tripdurmin'] > 120, 'mileage'] = (jan_df['tripdurmin'] / 60) * 14.9

In [13]:
# View results
jan_df.head()

,year,month,Trip Duration,tripdurmin,Start Time,starthour,weekday,Stop Time,Start Station ID,Start Station Name,...,End Station Longitude,distance,Bike ID,User Type,Birth Year,Gender,age,sgender,season,mileage
0,2017,Jan,680,11.333333,2017-01-01 00:00:21,00,Sunday,2017-01-01 00:11:41,3226,W 82 St & Central Park West,...,-73.976206,0.542775,25542,Subscriber,1965.0,2,52.0,Female,Winter,1.408356
1,2017,Jan,1282,21.366667,2017-01-01 00:00:45,00,Sunday,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,...,-73.988084,1.341340,21136,Subscriber,1987.0,2,30.0,Female,Winter,2.655164
2,2017,Jan,648,10.800000,2017-01-01 00:00:57,00,Sunday,2017-01-01 00:11:46,3143,5 Ave & E 78 St,...,-73.961199,0.576109,18147,Customer,NaN,0,NaN,Unknown,Winter,1.342080
3,2017,Jan,631,10.516667,2017-01-01 00:01:10,00,Sunday,2017-01-01 00:11:42,3143,5 Ave & E 78 St,...,-73.961199,0.576109,21211,Customer,NaN,0,NaN,Unknown,Winter,1.306871
4,2017,Jan,621,10.350000,2017-01-01 00:01:25,00,Sunday,2017-01-01 00:11:47,3143,5 Ave & E 78 St,...,-73.961199,0.576109,26819,Customer,NaN,0,NaN,Unknown,Winter,1.286160


### Explore the data to find obvious outliers or false data

Only for January

In [92]:
# View data frame information - How many trips have been recorded - Percentage of ridership growth
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726676 entries, 0 to 726675
Data columns (total 25 columns):
year                       726676 non-null int64
month                      726676 non-null object
Trip Duration              726676 non-null int64
tripdurmin                 726676 non-null float64
Start Time                 726676 non-null object
starthour                  726676 non-null object
weekday                    726676 non-null object
Stop Time                  726676 non-null object
Start Station ID           726676 non-null int64
Start Station Name         726676 non-null object
Start Station Latitude     726676 non-null float64
Start Station Longitude    726676 non-null float64
End Station ID             726676 non-null int64
End Station Name           726676 non-null object
End Station Latitude       726676 non-null float64
End Station Longitude      726676 non-null float64
distance                   726676 non-null float64
Bike ID                    726676 non

In [93]:
# Proportion of short-term customers and annual subscribers
jan_df['User Type'].value_counts()

Subscriber    700465
Customer       26211
Name: User Type, dtype: int64

In [94]:
# Peak hours in which bikes are used during the month
jan_df.starthour.value_counts()

08    73457
17    65859
18    62736
09    52596
16    47103
19    42829
15    42405
14    42158
07    41589
13    39937
12    37833
10    33992
11    33208
20    28683
21    19700
06    19491
22    14347
23     9198
05     5812
00     5547
01     3392
02     2015
04     1430
03     1359
Name: starthour, dtype: int64

In [95]:
# Number of unique start stations
jan_df['Start Station ID'].nunique()

609

In [96]:
# Number of trips from start stations
jan_df['Start Station ID'].value_counts()

519     8795
435     5454
497     5110
402     4913
490     4816
477     4480
285     4407
379     4150
459     4114
3255    3993
523     3977
509     3864
229     3809
284     3802
3263    3711
359     3704
168     3698
444     3692
520     3624
368     3523
3141    3508
492     3438
527     3403
504     3370
517     3360
472     3333
358     3201
505     3134
446     3117
442     3104
        ... 
3128     115
3123     112
3043     110
3179     101
3340      98
3120      96
3127      94
3327      94
298       93
3044      91
3053      85
3075      85
3342      75
3326      74
3393      74
255       68
3333      68
3395      67
3239      67
2005      62
3352      57
3394      46
3330      42
3245      35
3219      25
3240      19
3017       4
3446       2
3036       1
3040       1
Name: Start Station ID, Length: 609, dtype: int64

In [20]:
# Number of unique end stations
jan_df['End Station ID'].nunique()

612

In [21]:
# Number of arrivals to end stations
jan_df['End Station ID'].value_counts()

519     8791
402     5915
497     5679
435     5507
490     4882
520     4823
285     4573
477     4532
459     4491
3255    4205
379     4131
509     4112
523     4023
492     3945
229     3913
284     3887
3263    3827
168     3804
359     3753
368     3675
444     3544
3141    3495
382     3437
527     3398
426     3372
472     3337
504     3314
358     3195
482     3191
505     3153
        ... 
3043     101
443       98
3061      95
298       89
3333      80
3179      79
3393      79
3044      75
3053      73
3342      71
3352      68
3239      67
2005      65
3326      64
3075      63
255       57
3395      49
3330      47
3302      45
3394      45
3245      35
3219      28
3017      14
3240       9
3250       3
3446       2
3447       2
3040       1
3036       1
3183       1
Name: End Station ID, Length: 612, dtype: int64

In [97]:
# Gender breakdown of active participants - Gender (Zero=unknown; 1=male; 2=female) 
jan_df.sgender.value_counts()

Male       541849
Female     152992
Unknown     31835
Name: sgender, dtype: int64

In [98]:
# Average trip duration by age - Get the max and the min
# Min and Max Age = 2017 - birth year
print(f'Max age: {jan_df.age.max()}')
print(f'Min age: {jan_df.age.min()}')
print(f'Uniques: {jan_df.age.nunique()}')

Max age: 132.0
Min age: 17.0
Uniques: 85


In [99]:
# Average trip duration by age - Differents values for age
# Age = 2017 - birth year 
jan_df.age.value_counts()

37.0     48636
32.0     27156
29.0     26745
31.0     26543
28.0     26460
33.0     25520
30.0     25217
27.0     24776
34.0     24582
35.0     22105
26.0     21482
36.0     21044
38.0     18479
25.0     18095
47.0     17280
39.0     17200
41.0     16389
40.0     16286
43.0     15796
46.0     15710
48.0     15324
42.0     15296
45.0     15260
24.0     13972
44.0     13924
49.0     13670
53.0     12798
50.0     12652
52.0     12613
51.0     12540
         ...  
72.0       574
75.0       571
73.0       468
76.0       459
74.0       402
77.0       322
79.0       187
117.0       90
78.0        85
83.0        62
132.0       56
81.0        44
118.0       25
116.0       24
85.0        23
105.0       21
96.0        14
107.0       13
94.0        12
99.0        11
131.0       10
82.0        10
80.0        10
91.0        10
102.0        8
86.0         7
100.0        6
124.0        6
122.0        3
97.0         1
Name: age, Length: 85, dtype: int64

In [100]:
# Average trip duration by age
jan_df.groupby('age')['tripdurmin'].mean()

age
17.0     10.780345
18.0     10.182742
19.0     11.322078
20.0     12.306248
21.0     13.102444
22.0     12.023740
23.0     10.210682
24.0     12.399214
25.0     11.108418
26.0     11.399489
27.0     16.793854
28.0     10.727478
29.0     11.258855
30.0     11.210024
31.0     10.917126
32.0     11.460928
33.0     10.330312
34.0     11.855510
35.0     11.502646
36.0     12.272496
37.0     25.714336
38.0     11.405438
39.0     11.449595
40.0     11.569982
41.0     12.084505
42.0     11.888932
43.0     11.916686
44.0     12.328683
45.0     11.257503
46.0     11.940134
           ...    
72.0      9.712108
73.0     12.154024
74.0     12.162687
75.0     11.443812
76.0     11.868664
77.0     11.207246
78.0     11.189412
79.0     12.383155
80.0     14.160000
81.0     12.898106
82.0     14.635000
83.0     12.076613
85.0      6.755072
86.0     20.947619
91.0     11.683333
94.0      6.487500
96.0      3.250000
97.0     22.883333
99.0      7.784848
100.0     5.527778
102.0    59.502083
105.0   

In [28]:
# Average distance in miles that a bike is ridden - Uniques
jan_df['Bike ID'].nunique()

8176

In [29]:
# Average distance in miles that a bike is ridden
jan_df.groupby(['Bike ID']).agg({'distance': 'mean'}).sort_values(['distance'],ascending=False).head(10)

,distance
Bike ID,
20624,3.222267
22044,2.790730
24964,2.579924
24230,2.478572
26340,2.263342
16665,2.012900
22233,1.815943
22858,1.801996
19042,1.751008


In [30]:
# Which bikes (by ID) are most likely due for repair or inspection in the timespan?
jan_df.groupby(['Bike ID']).agg({'distance': 'sum'}).sort_values(['distance'],ascending=False).head(10)

,distance
Bike ID,
26785,319.839930
26323,312.144546
26022,311.877987
26603,289.426518
27018,285.901735
26585,284.286797
25969,278.830213
26604,278.450843
22756,277.365294


In [31]:
# Variability by bike ID
jan_df.groupby(['Bike ID']).agg({'distance': 'std'}).sort_values(['distance'],ascending=False).head(10)

,distance
Bike ID,
20624,2.358110
22233,2.063466
24823,1.995539
18997,1.826547
22858,1.777865
17787,1.694164
22044,1.663145
16582,1.611505
19556,1.562297


### __Make final data frames__

In [101]:
# Usertype + Gender
user_df = jan_df.groupby(['User Type', 'sgender']).agg({i:'count' for i in jan_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "Birth Year":"trips", "User Type" : "user type"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Jan')
user_df

,year,month,user type,gender,trips
0,2017,Jan,Customer,Female,1022
1,2017,Jan,Customer,Male,2371
2,2017,Jan,Customer,Unknown,22818
3,2017,Jan,Subscriber,Female,151970
4,2017,Jan,Subscriber,Male,539478
5,2017,Jan,Subscriber,Unknown,9017


In [102]:
# Trips by hour, season and weekday
season_df = jan_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in jan_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"Birth Year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Jan')
season_df

,year,month,starthour,season,weekday,trips
0,2017,Jan,00,Winter,Friday,890
1,2017,Jan,00,Winter,Monday,671
2,2017,Jan,00,Winter,Saturday,1111
3,2017,Jan,00,Winter,Sunday,1365
4,2017,Jan,00,Winter,Thursday,620
5,2017,Jan,00,Winter,Tuesday,431
6,2017,Jan,00,Winter,Wednesday,459
7,2017,Jan,01,Winter,Friday,441
8,2017,Jan,01,Winter,Monday,386
9,2017,Jan,01,Winter,Saturday,689


In [103]:
# Start Stations
stat_df = jan_df.groupby(['Start Station ID',]).agg({i:'count' for i in jan_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"Start Station ID" : "stationid", "Birth Year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Jan')
stat_df

,year,month,stationid,startrips
0,2017,Jan,72,1526
1,2017,Jan,79,1079
2,2017,Jan,82,541
3,2017,Jan,83,643
4,2017,Jan,116,1918
5,2017,Jan,119,156
6,2017,Jan,120,403
7,2017,Jan,127,2335
8,2017,Jan,128,2711
9,2017,Jan,143,1157


In [104]:
# End Stations
statend_df = jan_df.groupby(['End Station ID',]).agg({i:'count' for i in jan_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"End Station ID" : "stationid", "Birth Year":"endtrips"}, inplace=True)
statend_df

,stationid,endtrips
0,72,1486
1,79,1104
2,82,534
3,83,649
4,116,1939
5,119,160
6,120,351
7,127,2353
8,128,2862
9,143,1086


In [105]:
# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
stat_df.count()

year         609
month        609
stationid    612
startrips    609
endtrips     612
dtype: int64

In [106]:
# Age - Trip duration
agedur_df = jan_df.groupby(by=['age','User Type'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.rename(columns={"User Type" : "user type"}, inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Jan')
agedur_df

,year,month,age,user type,count,mean
0,2017,Jan,17.0,Subscriber,859,10.780345
1,2017,Jan,18.0,Customer,6,12.252778
2,2017,Jan,18.0,Subscriber,1799,10.175838
3,2017,Jan,19.0,Customer,114,21.845322
4,2017,Jan,19.0,Subscriber,1814,10.660750
5,2017,Jan,20.0,Customer,92,21.146014
6,2017,Jan,20.0,Subscriber,2773,12.012970
7,2017,Jan,21.0,Customer,157,32.235775
8,2017,Jan,21.0,Subscriber,3696,12.289692
9,2017,Jan,22.0,Customer,107,23.111526


In [107]:
# Bikes considering distance from start station to end station
bike_df = jan_df.groupby(by=['Bike ID'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"Bike ID": "bikeid",  "count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Jan')
bike_df

,year,month,bikeid,trips,totmiles,avgmiles,stdmiles
0,2017,Jan,14529,97,95.172780,0.981163,0.771285
1,2017,Jan,14530,54,57.989698,1.073883,0.717556
2,2017,Jan,14531,96,99.902576,1.040652,0.795374
3,2017,Jan,14532,57,64.015970,1.123087,0.875559
4,2017,Jan,14533,96,99.907684,1.040705,0.897507
5,2017,Jan,14534,33,29.295531,0.887743,0.870007
6,2017,Jan,14535,61,68.832014,1.128394,0.942858
7,2017,Jan,14536,47,52.399733,1.114888,0.733107
8,2017,Jan,14537,59,66.594836,1.128726,0.945362
9,2017,Jan,14539,2,1.916148,0.958074,0.439284


In [108]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = jan_df.groupby(by=['Bike ID'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"Bike ID": "bikeid", "count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Jan')
bikemil_df

,year,month,bikeid,trips,totmileage,avgmileage,stdmileage
0,2017,Jan,14529,97,128.771333,1.327540,0.974913
1,2017,Jan,14530,54,69.661822,1.290034,0.740286
2,2017,Jan,14531,96,267.606482,2.787568,12.468398
3,2017,Jan,14532,57,80.632498,1.414605,0.988851
4,2017,Jan,14533,96,134.514524,1.401193,1.087361
5,2017,Jan,14534,33,39.514729,1.197416,1.233014
6,2017,Jan,14535,61,246.268136,4.037183,18.838128
7,2017,Jan,14536,47,72.814053,1.549235,1.055970
8,2017,Jan,14537,59,103.292524,1.750721,2.016027
9,2017,Jan,14539,2,2.253369,1.126684,0.360266


### __Export final data frames to csv files for Tableau__

In [109]:
user_df.to_csv("./2017/user2017.csv", encoding="utf-8", index=False, header=True) 

season_df.to_csv("./2017/season2017.csv", encoding="utf-8", index=False, header=True)

stat_df.to_csv("./2017/station2017.csv", encoding="utf-8", index=False, header=True)

agedur_df.to_csv("./2017/agedur2017.csv", encoding="utf-8", index=False, header=True)

bike_df.to_csv("./2017/bike2017.csv", encoding="utf-8", index=False, header=True)

bikemil_df.to_csv("./2017/mileage2017.csv", encoding="utf-8", index=False, header=True)

### Make the same transformations from February to December

`February`

In [117]:
# Read file and print the number of records and data types
csv_name = "./2017/201702.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Feb2017 : " + str(month_df.count()))

Records Feb2017 : Trip Duration              791647
Start Time                 791647
Stop Time                  791647
Start Station ID           791647
Start Station Name         791647
Start Station Latitude     791647
Start Station Longitude    791647
End Station ID             791647
End Station Name           791647
End Station Latitude       791647
End Station Longitude      791647
Bike ID                    791647
User Type                  784067
Birth Year                 742324
Gender                     791647
dtype: int64


In [118]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Birth Year,6.230428
User Type,0.957497
Trip Duration,0.000000
Start Time,0.000000
Stop Time,0.000000
Start Station ID,0.000000
Start Station Name,0.000000
Start Station Latitude,0.000000
Start Station Longitude,0.000000
End Station ID,0.000000


In [119]:
# Replace missing values for birth year with the median by gender
month_df['Birth Year'] = month_df.groupby(['Gender'])['Birth Year'].transform(lambda x: x.fillna(x.median()))

# Replace missing values for user type as if they were Customers
month_df['User Type'] = month_df['User Type'].transform(lambda x: x.fillna("Customer"))

In [120]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Trip Duration,0.0
Start Time,0.0
Stop Time,0.0
Start Station ID,0.0
Start Station Name,0.0
Start Station Latitude,0.0
Start Station Longitude,0.0
End Station ID,0.0
End Station Name,0.0
End Station Latitude,0.0


In [121]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Feb')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Winter')
month_df.insert(24, 'mileage', 0)

In [122]:
# Transform the values for gender
month_df.loc[month_df['Gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['Gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['Gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['Birth Year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['Trip Duration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["Start Time"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["Start Time"]).dt.strftime('%A')

In [123]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['Start Station Latitude'],row['Start Station Longitude']), 
                         (row['End Station Latitude'], row['End Station Longitude'])),
                         axis=1)

In [124]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [125]:
# Usertype + Gender
user_df = month_df.groupby(['User Type', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "Birth Year":"trips", "User Type" : "user type"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Feb')

In [126]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"Birth Year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Feb')

In [127]:
# Start Stations
stat_df = month_df.groupby(['Start Station ID',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"Start Station ID" : "stationid", "Birth Year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Feb')
stat_df.count()

year         614
month        614
stationid    614
startrips    614
dtype: int64

In [128]:
# End Stations
statend_df = month_df.groupby(['End Station ID',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"End Station ID" : "stationid", "Birth Year":"endtrips"}, inplace=True)
statend_df.count()

stationid    619
endtrips     619
dtype: int64

In [129]:
# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [130]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','User Type'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Feb')

In [131]:
# Bikes
bike_df = month_df.groupby(by=['Bike ID'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Feb')

In [132]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['Bike ID'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Feb')

In [133]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`March`

In [134]:
# Read file and print the number of records and data types
csv_name = "./2017/201703.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Mar2017 : " + str(month_df.count()))

Records Mar2017 : Trip Duration              727665
Start Time                 727665
Stop Time                  727665
Start Station ID           727665
Start Station Name         727665
Start Station Latitude     727665
Start Station Longitude    727665
End Station ID             727665
End Station Name           727665
End Station Latitude       727665
End Station Longitude      727665
Bike ID                    727665
User Type                  722529
Birth Year                 694819
Gender                     727665
dtype: int64


In [136]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Birth Year,4.513890
User Type,0.705819
Trip Duration,0.000000
Start Time,0.000000
Stop Time,0.000000
Start Station ID,0.000000
Start Station Name,0.000000
Start Station Latitude,0.000000
Start Station Longitude,0.000000
End Station ID,0.000000


In [137]:
# Replace missing values for birth year with the median by gender
month_df['Birth Year'] = month_df.groupby(['Gender'])['Birth Year'].transform(lambda x: x.fillna(x.median()))

# Replace missing values for user type as if they were Customers
month_df['User Type'] = month_df['User Type'].transform(lambda x: x.fillna("Customer"))

In [138]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
Trip Duration,0.0
Start Time,0.0
Stop Time,0.0
Start Station ID,0.0
Start Station Name,0.0
Start Station Latitude,0.0
Start Station Longitude,0.0
End Station ID,0.0
End Station Name,0.0
End Station Latitude,0.0


In [139]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Mar')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Spring')
month_df.insert(24, 'mileage', 0)

In [140]:
# Transform the values for gender
month_df.loc[month_df['Gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['Gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['Gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['Birth Year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['Trip Duration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["Start Time"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["Start Time"]).dt.strftime('%A')

In [141]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['Start Station Latitude'],row['Start Station Longitude']), 
                         (row['End Station Latitude'], row['End Station Longitude'])),
                         axis=1)

In [142]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [143]:
# Usertype + Gender
user_df = month_df.groupby(['User Type', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "Birth Year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Mar')

In [144]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"Birth Year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Mar')

In [145]:
# Start Stations
stat_df = month_df.groupby(['Start Station ID',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"Start Station ID" : "stationid", "Birth Year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Mar')

In [146]:
# End Stations
statend_df = month_df.groupby(['End Station ID',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"End Station ID" : "stationid", "Birth Year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [147]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','User Type'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Mar')

In [148]:
# Bikes
bike_df = month_df.groupby(by=['Bike ID'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Mar')

In [149]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['Bike ID'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Mar')

In [150]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)
    
with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`April`

In [151]:
# Read file and print the number of records and data types
csv_name = "./2017/201704.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Apr2017 : " + str(month_df.count()))

Records Apr2017 : tripduration               1315404
starttime                  1315404
stoptime                   1315404
start station id           1315404
start station name         1315404
start station latitude     1315404
start station longitude    1315404
end station id             1315404
end station name           1315404
end station latitude       1315404
end station longitude      1315404
bikeid                     1315404
usertype                   1315404
birth year                 1146732
gender                     1315404
dtype: int64


In [152]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,12.822829
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [155]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [156]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [157]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Apr')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Spring')
month_df.insert(24, 'mileage', 0)

In [158]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [159]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [160]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [161]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Apr')

In [162]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Apr')

In [163]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Apr')

In [164]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [165]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Apr')

In [166]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Apr')

In [167]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Apr')

In [168]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`May`

In [169]:
# Read file and print the number of records and data types
csv_name = "./2017/201705.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records May2017 : " + str(month_df.count()))

Records May2017 : tripduration               1523268
starttime                  1523268
stoptime                   1523268
start station id           1523268
start station name         1523268
start station latitude     1523268
start station longitude    1523268
end station id             1523268
end station name           1523268
end station latitude       1523268
end station longitude      1523268
bikeid                     1523268
usertype                   1523268
birth year                 1350445
gender                     1523268
dtype: int64


In [170]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,11.345541
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [171]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [172]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [173]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'May')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Spring')
month_df.insert(24, 'mileage', 0)

In [174]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [175]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [176]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [177]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'May')

In [178]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'May')

In [179]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'May')

In [180]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [181]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'May')

In [182]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'May')

In [183]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'May')

In [184]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`June`

In [185]:
# Read file and print the number of records and data types
csv_name = "./2017/201706.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Jun2017 : " + str(month_df.count()))

Records Jun2017 : tripduration               1731594
starttime                  1731594
stoptime                   1731594
start station id           1731594
start station name         1731594
start station latitude     1731594
start station longitude    1731594
end station id             1731594
end station name           1731594
end station latitude       1731594
end station longitude      1731594
bikeid                     1731594
usertype                   1731594
birth year                 1539162
gender                     1731594
dtype: int64


In [186]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,11.112998
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [187]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [188]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [189]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Jun')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Summer')
month_df.insert(24, 'mileage', 0)

In [190]:
# Missing data in a trip station id not in catalog
month_df.loc[(month_df['end station id'] == 3248), :'distance']

,year,month,tripduration,tripdurmin,starttime,starthour,weekday,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,distance
1505508,2017,Jun,220,0,2017-06-27 14:46:06,0,0,2017-06-27 14:49:47,466,W 25 St & 6 Ave,40.743954,-73.991449,3248,LPI Facility,0.0,0.0,0
1506369,2017,Jun,528,0,2017-06-27 15:03:14,0,0,2017-06-27 15:12:03,237,E 11 St & 2 Ave,40.730473,-73.986724,3248,LPI Facility,0.0,0.0,0
1506429,2017,Jun,317,0,2017-06-27 15:04:16,0,0,2017-06-27 15:09:34,347,Greenwich St & W Houston St,40.728846,-74.008591,3248,LPI Facility,0.0,0.0,0


In [66]:
# Missing data in a trip station id not in catalog
month_df.loc[(month_df['end station id'] == 3470), :'distance']

,year,month,tripduration,tripdurmin,starttime,starthour,weekday,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,distance
1431399,2017,Jun,4911,0,2017-06-26 11:07:36,0,0,2017-06-26 12:29:28,3470,Gowanus Tech Station,40.669802,-73.994905,3470,Gowanus Tech Station,40.669802,-73.994905,0
1453343,2017,Jun,1646,0,2017-06-26 17:39:50,0,0,2017-06-26 18:07:17,3470,Gowanus Tech Station,40.669802,-73.994905,3470,Gowanus Tech Station,40.669802,-73.994905,0
1492360,2017,Jun,1263,0,2017-06-27 10:01:18,0,0,2017-06-27 10:22:21,3470,Gowanus Tech Station,40.669802,-73.994905,3470,Gowanus Tech Station,40.669802,-73.994905,0
1493546,2017,Jun,70,0,2017-06-27 10:29:33,0,0,2017-06-27 10:30:44,3470,Gowanus Tech Station,40.669802,-73.994905,3470,Gowanus Tech Station,40.669802,-73.994905,0
1494268,2017,Jun,371,0,2017-06-27 10:47:16,0,0,2017-06-27 10:53:28,3470,Gowanus Tech Station,40.669802,-73.994905,3470,Gowanus Tech Station,40.669802,-73.994905,0


In [191]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [192]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [193]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [194]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Jun')

In [195]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Jun')

In [196]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Jun')

In [197]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [200]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Jun')

In [201]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Jun')

In [202]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Jun')

In [203]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`July`

In [204]:
# Read file and print the number of records and data types
csv_name = "./2017/201707.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Jul2017 : " + str(month_df.count()))

Records Jul2017 : tripduration               1735599
starttime                  1735599
stoptime                   1735599
start station id           1735599
start station name         1735599
start station latitude     1735599
start station longitude    1735599
end station id             1735599
end station name           1735599
end station latitude       1735599
end station longitude      1735599
bikeid                     1735599
usertype                   1735599
birth year                 1507003
gender                     1735599
dtype: int64


In [205]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,13.171015
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [206]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [207]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [208]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Jul')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Summer')
month_df.insert(24, 'mileage', 0)

In [209]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [210]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [211]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [212]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Jul')

In [213]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Jul')

In [214]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Jul')

In [215]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [216]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Jul')

In [217]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Jul')

In [218]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Jul')

In [219]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`August`

In [221]:
# Read file and print the number of records and data types
csv_name = "./2017/201708.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Aug2017 : " + str(month_df.count()))

Records Aug2017 : tripduration               1816498
starttime                  1816498
stoptime                   1816498
start station id           1816498
start station name         1816498
start station latitude     1816498
start station longitude    1816498
end station id             1816498
end station name           1816498
end station latitude       1816498
end station longitude      1816498
bikeid                     1816498
usertype                   1816498
birth year                 1591782
gender                     1816498
dtype: int64


In [222]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,12.370837
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [223]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [224]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [226]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Aug')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Summer')
month_df.insert(24, 'mileage', 0)

In [227]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [228]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [229]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [230]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Aug')

In [231]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Aug')

In [232]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Aug')

In [233]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [234]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Aug')

In [235]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Aug')

In [236]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Aug')

In [237]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`September`

In [238]:
# Read file and print the number of records and data types
csv_name = "./2017/201709.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Sep2017 : " + str(month_df.count()))

Records Sep2017 : tripduration               1878098
starttime                  1878098
stoptime                   1878098
start station id           1878098
start station name         1878098
start station latitude     1878098
start station longitude    1878098
end station id             1878098
end station name           1878098
end station latitude       1878098
end station longitude      1878098
bikeid                     1878098
usertype                   1878098
birth year                 1670630
gender                     1878098
dtype: int64


In [239]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,11.046708
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [240]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [241]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [242]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Sep')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Autumn')
month_df.insert(24, 'mileage', 0)

In [68]:
# Missing data in a trip station id not in catalog
month_df.loc[(month_df['end station id'] == 3188), :'distance']

,year,month,tripduration,tripdurmin,starttime,starthour,weekday,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,distance
1673107,2017,Sep,451063,0,2017-09-27 18:23:12,0,0,2017-10-02 23:40:55,519,Pershing Square North,40.751873,-73.977706,3188,NJCU,40.710109,-74.085849,0


In [69]:
# Missing data in a trip station id not in catalog
month_df.loc[(month_df['end station id'] == 3471), :'distance']

,year,month,tripduration,tripdurmin,starttime,starthour,weekday,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,distance
1311163,2017,Sep,724,0,2017-09-22 11:29:20,0,0,2017-09-22 11:41:24,3458,W 55 St & 6 Ave,40.763094,-73.97835,3471,NYCBS Depot - GOW Annex,40.668884,-73.994814,0


In [243]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [244]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [245]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [246]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Sep')

In [247]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Sep')

In [248]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Sep')

In [249]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [250]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Sep')

In [251]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Sep')

In [252]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Sep')

In [253]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`October`

In [254]:
# Read file and print the number of records and data types
csv_name = "./2017/201710.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Oct2017 : " + str(month_df.count()))

Records Oct2017 : tripduration               1897592
starttime                  1897592
stoptime                   1897592
start station id           1897592
start station name         1897592
start station latitude     1897592
start station longitude    1897592
end station id             1897592
end station name           1897592
end station latitude       1897592
end station longitude      1897592
bikeid                     1897592
usertype                   1897592
birth year                 1701999
gender                     1897592
dtype: int64


In [255]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,10.307432
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [256]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [257]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [258]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Oct')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Autumn')
month_df.insert(24, 'mileage', 0)

In [259]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [260]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [261]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [262]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Oct')

In [263]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Oct')

In [264]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Oct')

In [265]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [266]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Oct')

In [267]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Oct')

In [268]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Oct')

In [269]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`November`

In [270]:
# Read file and print the number of records and data types
csv_name = "./2017/201711.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Nov2017 : " + str(month_df.count()))

Records Nov2017 : tripduration               1330649
starttime                  1330649
stoptime                   1330649
start station id           1330649
start station name         1330649
start station latitude     1330649
start station longitude    1330649
end station id             1330649
end station name           1330649
end station latitude       1330649
end station longitude      1330649
bikeid                     1330649
usertype                   1330649
birth year                 1241304
gender                     1330649
dtype: int64


In [271]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,6.714393
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [272]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [273]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [274]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Nov')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Autumn')
month_df.insert(24, 'mileage', 0)

In [275]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [276]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [277]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [278]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Nov')

In [279]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Nov')

In [280]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Nov')

In [281]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [282]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Nov')

In [283]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Nov')

In [284]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Nov')

In [285]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

`December`

In [286]:
# Read file and print the number of records and data types
csv_name = "./2017/201712.csv"
month_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Dec2017 : " + str(month_df.count()))

Records Dec2017 : tripduration               889967
starttime                  889967
stoptime                   889967
start station id           889967
start station name         889967
start station latitude     889967
start station longitude    889967
end station id             889967
end station name           889967
end station latitude       889967
end station longitude      889967
bikeid                     889967
usertype                   889967
birth year                 850522
gender                     889967
dtype: int64


In [287]:
# Find missing values to define what to do with them
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
birth year,4.432187
tripduration,0.000000
starttime,0.000000
stoptime,0.000000
start station id,0.000000
start station name,0.000000
start station latitude,0.000000
start station longitude,0.000000
end station id,0.000000
end station name,0.000000


In [288]:
# Replace missing values for birth year with the median by gender
month_df['birth year'] = month_df.groupby(['gender'])['birth year'].transform(lambda x: x.fillna(x.median()))

In [289]:
# Verify that there are not more missing values
percent_missing = month_df.isnull().sum() * 100 / len(month_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,percent_missing
tripduration,0.0
starttime,0.0
stoptime,0.0
start station id,0.0
start station name,0.0
start station latitude,0.0
start station longitude,0.0
end station id,0.0
end station name,0.0
end station latitude,0.0


In [290]:
# Add columns to make transformations
month_df.insert(0, 'year', 2017)
month_df.insert(1, 'month', 'Dec')
month_df.insert(3, 'tripdurmin', 0)
month_df.insert(5, 'starthour', 0)
month_df.insert(6, 'weekday', 0)
month_df.insert(16, 'distance', 0)
month_df.insert(21, 'age', 0)
month_df.insert(22, 'sgender', '')
month_df.insert(23, 'season', 'Winter')
month_df.insert(24, 'mileage', 0)

In [291]:
# Transform the values for gender
month_df.loc[month_df['gender'] == 0, 'sgender'] = 'Unknown'
month_df.loc[month_df['gender'] == 1, 'sgender'] = 'Male'
month_df.loc[month_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person considering that the year was 2017
month_df['age'] = 2017 - month_df['birth year']

# Transform the durantion of the trip from seconds to minutes
month_df['tripdurmin'] = month_df['tripduration'] / 60

# Extact the hour fron the startime
month_df['starthour'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%H')

# Calculate the weekday fron the startime
month_df['weekday'] = pd.to_datetime(month_df["starttime"]).dt.strftime('%A')

In [292]:
# Calcularte the distance in miles from start station to end station
month_df['distance'] = month_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [293]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
month_df.loc[month_df['tripdurmin'] <= 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 7.456

# Trips over two hours max-out at 14.9 miles
month_df.loc[month_df['tripdurmin'] > 120, 'mileage'] = (month_df['tripdurmin'] / 60) * 14.9

In [294]:
# Usertype + Gender
user_df = month_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in month_df.columns[19:20]})
user_df.reset_index(inplace=True)
user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
user_df.insert(0, 'year', 2017)
user_df.insert(1, 'month', 'Dec')

In [295]:
# Trips by hour, season and weekday
season_df = month_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in month_df.columns[19:20]})
season_df.reset_index(inplace=True)
season_df.rename(columns={"birth year":"trips"}, inplace=True)
season_df.insert(0, 'year', 2017)
season_df.insert(1, 'month', 'Dec')

In [296]:
# Start Stations
stat_df = month_df.groupby(['start station id',]).agg({i:'count' for i in month_df.columns[19:20]})
stat_df.reset_index(inplace=True)
stat_df.rename(columns={"start station id" : "stationid", "birth year":"startrips"}, inplace=True)
stat_df.insert(0, 'year', 2017)
stat_df.insert(1, 'month', 'Dec')

In [297]:
# End Stations
statend_df = month_df.groupby(['end station id',]).agg({i:'count' for i in month_df.columns[19:20]})
statend_df.reset_index(inplace=True)
statend_df.rename(columns={"end station id" : "stationid", "birth year":"endtrips"}, inplace=True)

# Merge the start trips with the end trips by station id
stat_df = stat_df.merge(statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))

In [298]:
# Age - Trip duration
agedur_df = month_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
agedur_df.reset_index(inplace=True)
agedur_df.insert(0, 'year', 2017)
agedur_df.insert(1, 'month', 'Dec')

In [299]:
# Bikes
bike_df = month_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
bike_df.reset_index(inplace=True)
bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
bike_df.insert(0, 'year', 2017)
bike_df.insert(1, 'month', 'Dec')

In [300]:
# Bikes considering trip duration to calculate the mileage 
bikemil_df = month_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
bikemil_df.reset_index(inplace=True)
bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
bikemil_df.insert(0, 'year', 2017)
bikemil_df.insert(1, 'month', 'Dec')

In [301]:
with open('./2017/user2017.csv', 'a') as f:
    user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/season2017.csv', 'a') as f:
    season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/station2017.csv', 'a') as f:
    stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/agedur2017.csv', 'a') as f:
    agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/bike2017.csv', 'a') as f:
    bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2017/mileage2017.csv', 'a') as f:
    bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

### __Stations with trips but not in the JSON file (catalog)__

In [56]:
# Read file and print the number of records and data types
csv_name = "./2017/station2017.csv"
stations_df = pd.read_csv(csv_name, low_memory=False) 
csv_name = "./2017/stationsny.csv"
catalogue_df = pd.read_csv(csv_name, low_memory=False)

In [57]:
# Merge data
stations_df = stations_df.merge(catalogue_df, how="outer", left_on='stationid', right_on='Id', suffixes=('_left', '_right'))

In [58]:
# View results
stations_df.head()

,year,month,stationid,startrips,endtrips,Número de registros,Número de registros por stations.json,Altitude,Available Bikes,Available Docks,...,Postal Code,stAddress1,stAddress2,stationBeanList Índice (generado),Station Name,Status Key,Status Value,Test Station,Total Docks,Índice Del Documento (Generado)
0,2017.0,Jan,72.0,1470.0,1431.0,1.0,1.0,NaN,9.0,44.0,...,NaN,W 52 St & 11 Ave,NaN,21.0,W 52 St & 11 Ave,1.0,In Service,Falso,55.0,1.0
1,2017.0,Feb,72.0,1573.0,1519.0,1.0,1.0,NaN,9.0,44.0,...,NaN,W 52 St & 11 Ave,NaN,21.0,W 52 St & 11 Ave,1.0,In Service,Falso,55.0,1.0
2,2017.0,Mar,72.0,1821.0,1846.0,1.0,1.0,NaN,9.0,44.0,...,NaN,W 52 St & 11 Ave,NaN,21.0,W 52 St & 11 Ave,1.0,In Service,Falso,55.0,1.0
3,2017.0,Apr,72.0,2894.0,2873.0,1.0,1.0,NaN,9.0,44.0,...,NaN,W 52 St & 11 Ave,NaN,21.0,W 52 St & 11 Ave,1.0,In Service,Falso,55.0,1.0
4,2017.0,May,72.0,3320.0,3255.0,1.0,1.0,NaN,9.0,44.0,...,NaN,W 52 St & 11 Ave,NaN,21.0,W 52 St & 11 Ave,1.0,In Service,Falso,55.0,1.0


In [59]:
# Create an array with the stations ids from trips
stat_trip = stations_df['stationid'].unique()

In [60]:
# Create an array with the stations ids from catalog
stat_cat = stations_df['Id'].unique()

In [63]:
print("Unique id of stations that are not in the catalog:")
print(np.setdiff1d(stat_trip, stat_cat))

Unique id of stations that are not in the catalog:
[ 147.  153.  225.  255.  348.  352.  367.  384.  395.  444.  475.  488.
  511. 2004. 3017. 3098. 3111. 3114. 3188. 3197. 3219. 3230. 3237. 3246.
 3247. 3248. 3253. 3257. 3264. 3265. 3353. 3436. 3445. 3446. 3450. 3464.
 3470. 3471. 3476. 3480. 3548. 3550. 3633. 3636.   nan]
